In [55]:
# Import necessary packages
from sqlalchemy import create_engine, inspect
from sqlalchemy.exc import SQLAlchemyError
from langchain import LLMChain
from langchain_ollama.llms import OllamaLLM
from langchain.prompts import PromptTemplate
from langchain_core.vectorstores import VectorStoreRetriever
from langchain.vectorstores import FAISS
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
open_ai_key = ""
db_user = "root"
db_password = "password"
db_host = "localhost"
db_name = "otl_temp"

In [74]:
# 1. Set up the database connection
DATABASE_URL = f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}"
engine = create_engine(DATABASE_URL)
inspector = inspect(engine)

# 2. Retrieve and format database schema information
def get_database_schema():
    schema_info = {}
    for table_name in inspector.get_table_names():
        columns = inspector.get_columns(table_name)
        schema_info[table_name] = [column['name'] for column in columns]
    
    schema_str = "Database Schema:\n"
    for table, columns in schema_info.items():
        schema_str += f"Table: {table}\nColumns: {', '.join(columns)}\n\n"
    return schema_info, schema_str
# Get the schema information and schema string
schema_info, database_schema = get_database_schema()  # Convert schema information into a dictionary and a string

In [75]:
table_list = [
    'import_pi', 'supplier','import_flow_status','country','supp_org_type'
]

filtered_schema_info = {table: columns for table, columns in schema_info.items() if table in table_list}
filtered_schema_info

{'country': ['id', 'country'],
 'import_flow_status': ['id', 'status'],
 'import_pi': ['id',
  'unique_id',
  'imp_pi_number',
  'revision_date',
  'imp_pi_rev_type_id',
  'imp_pi_date',
  'supplier_id',
  'cfr_value',
  'freight_charge',
  'fob',
  'etd_date_china',
  'imp_pi_status_id',
  'cancellation_date',
  'cancellation_reason',
  'import_flow_status_id',
  'isCompensationAdjusted',
  'total_revision_count',
  'submitted_to_commercial',
  'created_at',
  'created_by',
  'updated_at',
  'updated_by'],
 'supp_org_type': ['id', 'org_type'],
 'supplier': ['id',
  'name',
  'short_name',
  'address',
  'active',
  'contact_person',
  'contact_number',
  'email',
  'website',
  'province',
  'country_id',
  'port',
  'enlisted_date',
  'no_of_factories',
  'total_employee',
  'annual_revenue',
  'supp_org_type_id',
  'created_by',
  'created_at',
  'updated_at',
  'updated_by']}

In [58]:
schema_information = """
# Schema Information

## Table: country
- **Columns:**
    - id: INT, Primary Key. Unique identifier for each country.
    - country: VARCHAR. Name of the country.
- **Description:**
    The `country` table stores information about different countries. It is primarily used for referencing country information in other tables, such as suppliers. 
- **Relationships:**
    - This table can be referenced by other tables (e.g., `supplier`) through the `country_id` column, establishing a many-to-one relationship.

## Table: import_flow_status
- **Columns:**
    - id: INT, Primary Key. Unique identifier for each import flow status.
    - status: VARCHAR. Descriptive status name (e.g., Pending, Approved).
- **Description:**
    The `import_flow_status` table maintains the various stages of the import flow process. It is used to track and categorize the current status of an import process in the `import_pi` table.
- **Relationships:**
    - This table is referenced by the `import_pi` table through the `import_flow_status_id` column, indicating the status of the import process.

## Table: import_pi
- **Columns:**
    - id: INT, Primary Key. Unique identifier for each import Proforma Invoice (PI).
    - unique_id: VARCHAR. Unique identifier for the PI within the import process.
    - imp_pi_number: VARCHAR. Number assigned to the Import PI.
    - revision_date: DATE. Date when the PI was last revised.
    - imp_pi_rev_type_id: INT. Type of revision made to the PI.
    - imp_pi_date: DATE. Date of the PI issuance.
    - supplier_id: INT, Foreign Key (supplier.id). References the supplier providing the goods in the PI.
    - cfr_value: DECIMAL. Cost and Freight value specified in the PI.
    - freight_charge: DECIMAL. Freight charges associated with the PI.
    - fob: DECIMAL. Free on Board value specified in the PI.
    - etd_date_china: DATE. Estimated Time of Departure from China.
    - imp_pi_status_id: INT. Status ID indicating the current state of the PI.
    - cancellation_date: DATE. Date when the PI was cancelled, if applicable.
    - cancellation_reason: VARCHAR. Reason for cancellation, if any.
    - import_flow_status_id: INT, Foreign Key (import_flow_status.id). References the current status of the import process.
    - isCompensationAdjusted: BOOLEAN. Indicates whether compensation adjustments have been made.
    - total_revision_count: INT. Number of times the PI has been revised.
    - submitted_to_commercial: BOOLEAN. Indicates if the PI has been submitted to the commercial department.
    - created_at: TIMESTAMP. Date and time when the record was created.
    - created_by: INT. User ID of the person who created the record.
    - updated_at: TIMESTAMP. Date and time when the record was last updated.
    - updated_by: INT. User ID of the person who last updated the record.
- **Description:**
    The `import_pi` table captures detailed information about import Proforma Invoices (PIs), including financial values (CFR, FOB), status, and supplier information. Each record represents a unique PI in the import process.
- **Relationships:**
    - The table references `supplier` through the `supplier_id` column, indicating the supplier associated with each PI.
    - It also references `import_flow_status` through the `import_flow_status_id` column to indicate the status of the import process.
    - Many-to-One relationships are established with `supplier` and `import_flow_status`, meaning multiple records in `import_pi` can relate to a single record in these tables.

## Table: supp_org_type
- **Columns:**
    - id: INT, Primary Key. Unique identifier for each supplier organization type.
    - org_type: VARCHAR. Type of the organization (e.g., Manufacturer, Distributor).
- **Description:**
    The `supp_org_type` table classifies suppliers based on their organization type, such as whether they are manufacturers, distributors, or other types of organizations.
- **Relationships:**
    - This table is referenced by the `supplier` table through the `supp_org_type_id` column, linking each supplier to its organizational type.

## Table: supplier
- **Columns:**
    - id: INT, Primary Key. Unique identifier for each supplier.
    - name: VARCHAR. Full name of the supplier.
    - short_name: VARCHAR. Short or abbreviated name of the supplier.
    - address: TEXT. Detailed address of the supplier.
    - active: BOOLEAN. Indicates whether the supplier is currently active.
    - contact_person: VARCHAR. Name of the contact person at the supplier organization.
    - contact_number: VARCHAR. Contact number of the supplier or contact person.
    - email: VARCHAR. Email address for communication.
    - website: VARCHAR. Supplier's website URL.
    - province: VARCHAR. Province or state where the supplier is located.
    - country_id: INT, Foreign Key (country.id). References the country where the supplier is located.
    - port: VARCHAR. Port used by the supplier for shipping goods.
    - enlisted_date: DATE. Date when the supplier was enlisted or registered.
    - no_of_factories: INT. Number of factories owned by the supplier.
    - total_employee: INT. Total number of employees working for the supplier.
    - annual_revenue: DECIMAL. Annual revenue of the supplier.
    - supp_org_type_id: INT, Foreign Key (supp_org_type.id). References the type of organization the supplier belongs to.
    - created_by: INT. User ID of the person who created the record.
    - created_at: TIMESTAMP. Date and time when the record was created.
    - updated_at: TIMESTAMP. Date and time when the record was last updated.
    - updated_by: INT. User ID of the person who last updated the record.
- **Description:**
    The `supplier` table contains detailed information about suppliers, such as contact information, location, and organization type. Each supplier is uniquely identified and can be linked to import Proforma Invoices (PIs) and other records in the import process.
- **Relationships:**
    - The table references `country` through the `country_id` column, indicating the country of the supplier.
    - It references `supp_org_type` through the `supp_org_type_id` column, linking the supplier to its organizational type.
    - This table can be referenced by other tables such as `import_pi` through the `supplier_id` column, establishing a many-to-one relationship where multiple PIs can be associated with a single supplier.
"""
details_information = str(filtered_schema_info) + "\n" + schema_information


In [59]:
from langchain_huggingface import HuggingFaceEmbeddings
#embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-large",
    api_key = open_ai_key
)
vectorstore = FAISS.from_texts(details_information, embeddings)
retriever = vectorstore.as_retriever()

In [60]:
few_shot_examples = """
Example 1:
User Request: Show all active suppliers along with their contact details, country, and organization type.
SQL Query: 
SELECT 
    supplier.id AS supplier_id, 
    supplier.name AS supplier_name, 
    supplier.contact_person, 
    supplier.contact_number, 
    supplier.email, 
    country.country AS country_name, 
    supp_org_type.org_type AS organization_type
FROM supplier
LEFT JOIN country ON supplier.country_id = country.id
LEFT JOIN supp_org_type ON supplier.supp_org_type_id = supp_org_type.id
WHERE supplier.active = TRUE;

Example 2:
User Request: Display all cancelled Proforma Invoices (PIs) along with the supplier name and cancellation reason.
SQL Query: 
SELECT 
    import_pi.imp_pi_number, 
    import_pi.cancellation_date, 
    import_pi.cancellation_reason, 
    supplier.name AS supplier_name
FROM import_pi
JOIN supplier ON import_pi.supplier_id = supplier.id
WHERE import_pi.cancellation_date IS NOT NULL;

Example 3:
User Request: Show the total number of Proforma Invoices (PIs) created by each supplier.
SQL Query: 
SELECT 
    supplier.name AS supplier_name, 
    COUNT(import_pi.id) AS total_pis
FROM supplier
LEFT JOIN import_pi ON supplier.id = import_pi.supplier_id
GROUP BY supplier.name;

Example 4:
User Request: List all Proforma Invoices along with their import flow status.
SQL Query: 
SELECT 
    import_pi.imp_pi_number, 
    import_pi.imp_pi_date, 
    import_flow_status.status AS import_flow_status
FROM import_pi
JOIN import_flow_status ON import_pi.import_flow_status_id = import_flow_status.id;

Example 5:
User Request: Find all suppliers who do not have any associated Proforma Invoices.
SQL Query: 
SELECT 
    supplier.id AS supplier_id, 
    supplier.name AS supplier_name, 
    supplier.contact_person, 
    supplier.contact_number
FROM supplier
LEFT JOIN import_pi ON supplier.id = import_pi.supplier_id
WHERE import_pi.id IS NULL;

Example 6:
User Request: Get the total CFR value of all Proforma Invoices (PIs) for each supplier.
SQL Query: 
SELECT 
    supplier.name AS supplier_name, 
    SUM(import_pi.cfr_value) AS total_cfr_value
FROM import_pi
JOIN supplier ON import_pi.supplier_id = supplier.id
GROUP BY supplier.name;

Example 7:
User Request: Show all import Proforma Invoices (PIs) that have been submitted to the commercial department.
SQL Query: 
SELECT 
    import_pi.imp_pi_number, 
    import_pi.imp_pi_date, 
    supplier.name AS supplier_name
FROM import_pi
JOIN supplier ON import_pi.supplier_id = supplier.id
WHERE import_pi.submitted_to_commercial = TRUE;

Example 8:
User Request: List all Proforma Invoices with more than 5 revisions along with supplier name and total revisions count.
SQL Query: 
SELECT 
    import_pi.imp_pi_number, 
    import_pi.imp_pi_date, 
    supplier.name AS supplier_name, 
    import_pi.total_revision_count
FROM import_pi
JOIN supplier ON import_pi.supplier_id = supplier.id
WHERE import_pi.total_revision_count > 5;

Example 9:
User Request: Display the shipment details for each Import CI along with the invoice number and the shipment status.
SQL Query: 
SELECT 
    import_shipment.id AS shipment_id, 
    import_shipment.import_ci_id, 
    import_ci.invoice_no, 
    import_shipment.actual_ship_date, 
    import_shipment.eta_ctg_date, 
    IF(inventory.mark_as_full_shipment = 1, 'Full Shipment', 'Partial Shipment') AS shipment_status
FROM import_shipment
LEFT JOIN import_ci ON import_shipment.import_ci_id = import_ci.id
LEFT JOIN inventory ON inventory.import_ci_id = import_shipment.import_ci_id;

Example 10:
User Request: List all suppliers along with the total number of employees and the country they belong to.
SQL Query: 
SELECT 
    supplier.name AS supplier_name, 
    supplier.total_employee, 
    country.country AS country_name
FROM supplier
JOIN country ON supplier.country_id = country.id;
"""

In [61]:
llm = ChatOpenAI(model_name="gpt-4o", openai_api_key=open_ai_key)

In [62]:
text = f"""You are provided with the following database schema information and user request. 
Your task is to generate a valid SQL query that only uses the tables and columns listed in the provided schema.

    Ensure that:
    1. You **only** use tables and columns that exist in the provided schema.
    2. If the user request cannot be fulfilled with the available schema, respond with "The requested information is not available in the current schema."
    3. Avoid guessing table or column names that are not explicitly mentioned in the schema.
    4. Use the most relevant tables and columns to generate the query.

    Here are some examples of valid SQL queries:
    {few_shot_examples}"""


prompt_template = PromptTemplate(
    template=f"""
    {text}

    User Request: {{request}}
    
    SQL Query:
    """,
    input_variables=["context", "request"]
)

In [67]:
def generate_sql_query_with_rag(user_request):
    retrieved_docs = retriever.invoke(user_request)
   
    relevant_context = "\n".join([doc.page_content for doc in retrieved_docs])

    prompt = prompt_template.format(context=relevant_context, request=user_request)

    sql_query = llm.invoke(prompt)
    
    return sql_query

In [64]:
from sqlalchemy.sql import text  # Import the text function

def execute_sql_query(sql_query):
    if not isinstance(sql_query, str):
        return f"Error: The provided SQL query is not a string. Got {type(sql_query)}"

    try:
        with engine.connect() as connection:
            result = connection.execute(text(sql_query))
            return result.fetchall()
    except SQLAlchemyError as e:
        print("*****SQL Error****")
        return str(e)

In [65]:
import re

def extract_sql_query(text):
    pattern = r"```sql(.*?)```"
    match = re.search(pattern, text, re.DOTALL)  # re.DOTALL to match across multiple lines
    
    if match:
        sql_query = match.group(1).strip()
        formatted_query = " ".join(sql_query.split())
        return formatted_query
    else:
        return None

In [86]:
user_prompt = "Show me the list of import's all information of supplier which short name is AB"
generated_sql = generate_sql_query_with_rag(user_prompt)
query = extract_sql_query(generated_sql.content)

print("----------------------------------------")
print("Query:\n", query)
print("----------------------------")

result = execute_sql_query(query)
result

----------------------------------------
Query:
 SELECT import_pi.*, supplier.* FROM import_pi JOIN supplier ON import_pi.supplier_id = supplier.id WHERE supplier.short_name = 'AB';
----------------------------


[(2, '1697475925', 'APBL/87803/0051012022', None, None, datetime.date(2023, 1, 24), 1, 4040.0, 0.0, 4040.0, datetime.date(2023, 2, 13), 1, None, None, 5, 0, 0, 1, datetime.datetime(2023, 2, 6, 17, 6, 2), 70, datetime.datetime(2023, 2, 7, 12, 14, 33), 70, 1, 'Allplast Bangladesh Ltd', 'AB', 'Mulgaon/Sandan Para/Kaligonj/Gazipur', 1, 'Mr.. Mamun', '8801844659762', 'apbl49@rflgroupbd.com', 'http://apblbd.com', None, 1, None, datetime.date(2017, 5, 18), 2, 650, 10.0, 1, None, None, datetime.datetime(2023, 12, 21, 15, 28, 48), 33),
 (3, '1697219019', 'APBL/87803/00052022022', None, None, datetime.date(2023, 2, 1), 1, 10800.0, 0.0, 10800.0, datetime.date(2023, 2, 14), 1, None, None, 2, 0, 0, 1, datetime.datetime(2023, 2, 7, 12, 31, 53), 70, None, None, 1, 'Allplast Bangladesh Ltd', 'AB', 'Mulgaon/Sandan Para/Kaligonj/Gazipur', 1, 'Mr.. Mamun', '8801844659762', 'apbl49@rflgroupbd.com', 'http://apblbd.com', None, 1, None, datetime.date(2017, 5, 18), 2, 650, 10.0, 1, None, None, datetime.dateti